# A noptebook to get a table with AS3D, UBERON ID, and Sex

# Import libraries

In [54]:
import requests
import pandas as pd
from io import StringIO

# Write SPARQL query and make request

In [55]:
query = '''PREFIX ccf: <http://purl.org/ccf/>
PREFIX HRA: <https://purl.humanatlas.io/collection/hra>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
PREFIX FMA: <http://purl.org/sig/ont/fma/fma>

SELECT DISTINCT ?as_label ?uberon_id ?sex ?ref_organ
FROM HRA:
FROM ccf:
WHERE {
  ?as_3d ccf:has_reference_organ ?ref_organ ;
		rdfs:label ?as_label_long ;
		ccf:organ_owner_sex ?sex;
		ccf:representation_of ?as_id .
	
	# ?x ccf:ccf_pref_label ?organ_label ;
	#  ?ref_organ ccf:representation_of ?organIri .
	#  ?organ_iri ccf:ccf_pref_label ?organ_label .

  FILTER(!CONTAINS(STR(?ref_organ), '/united-'))
  FILTER (str(?as_3d) != str(?ref_organ))
	BIND(REPLACE(REPLACE(STR(?as_id), STR(UBERON:), 'UBERON:'), STR(FMA:), 'FMA:') as ?uberon_id)
	BIND(REPLACE(STR(?as_label_long), STR("Spatial entity of "), '') as ?as_label)
}
'''

# set endpoint
sparql_endpoint = "http://lod.humanatlas.io/sparql"

# Set up the parameters for the request
params = {
    "query": query,
}

headers = {
    "Accept": "text/csv",
    # "Content-Type": "application/x-www-form-urlencoded"
}

# Send the request to the SPARQL endpoint
response = requests.get(sparql_endpoint, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Read the CSV response into a pandas DataFrame
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)
    # Print the DataFrame
    print(df)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

                                     as_label       uberon_id     sex  \
0     female abdominal ostium of uterine tube       FMA:77049  Female   
1              female anterior wall of uterus      FMA:224876  Female   
2                       female body of uterus  UBERON:0009853  Female   
3                               female cervix  UBERON:0000002  Female   
4                 female external cervical os  UBERON:0013760  Female   
...                                       ...             ...     ...   
1168       male long posterior ciliary artery       FMA:49957    Male   
1169                male marginal artery of D       FMA:14824    Male   
1170                  male median sacral vein  UBERON:0006687    Male   
1171                 male middle cardiac vein  UBERON:0009687    Male   
1172                 male middle colic artery  UBERON:0001196    Male   

                                              ref_organ  
0     https://purl.humanatlas.io/ref-organ/uterus-fe...  
1     h

# Export result to CSV

In [56]:
df.to_csv("output/as3d_uberon_sex.csv", index=False)